In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import HostCuts as H
from astropy.table import join
from SAGA.utils import join_str_arr

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
hosts = saga.host_catalog.load(include_stats=True, query="paper2_complete")
print(len(hosts))

In [ ]:
hosts['phot_coverage'] = join_str_arr(
    np.where(Query('COVERAGE_SDSS > 0.95').mask(hosts), 'S', '-'),
    np.where(Query('COVERAGE_DES_DR1 > 0.95').mask(hosts), 'D', '-'),
    np.where(Query('(COVERAGE_DECALS_DR6 > 0.95) | (COVERAGE_DECALS_DR7 > 0.95)').mask(hosts), 'L', '-'),
)
hosts['sats_below'] = np.where(hosts['sats_Mr_limit'] == hosts['sats_total'], '', np.char.mod('(%d)', hosts['sats_total']-hosts['sats_Mr_limit']))
hosts["spec_coverage"] = 100*(1-hosts["paper2_need_spec"] / hosts["paper2_total"])
hosts['internal_name'] = np.where(hosts["SAGA_NAME"] != "", hosts["SAGA_NAME"], hosts["HOSTID"])

In [ ]:
t = hosts['COMMON_NAME', 'internal_name', 'RA', "DEC", 'V_HELIO', 'DIST', 'K_ABS', 'phot_coverage', 'spec_coverage', 'sats_Mr_limit', 'sats_below', 'sats_missed_corrected']

t.sort('RA')

t['RA'].format = '%.3f'
t['DEC'].format = '%.3f'
t['V_HELIO'].format = '%d'
t['DIST'].format = '%.1f'
t['K_ABS'].format = '%.2f'
t['sats_missed_corrected'].format = '%.2f'
t['spec_coverage'].format = '%.1f'

In [ ]:
colheads = [
    "Common\nName",
    "SAGA\nName",
    "R.A.\n[deg]",
    "Dec.\n[deg]",
    "HRV\n[\\kms]",
    "$D$\n[Mpc]",
    "$M_{K,o}$\n[mag]",
    "Phot.\nCoverage",
    "Spec.\nCoverage (\%)",
    "Confirmed\n$N_\\text{sat}$",
    "Miss.\n$N_\\text{sat}$"
]

In [ ]:
colheads_formatted = [[], [], []]
for i, colhead in enumerate(colheads):
    colhead = colhead.splitlines()
    while len(colhead) < 2:
        colhead.insert(0, "~")
    colhead.append(f"({i+1})")
    for new, l in zip(colheads_formatted, colhead):
        new.append(l)

In [ ]:
print("\\\\[-6pt]\n".join((" & ".join((f"\colhead{{{i}}}" for i in row)) for row in colheads_formatted)))

In [ ]:
from io import StringIO
o = StringIO()
t.write(o, format="ascii.aastex")
o.seek(0)

for line in o:
    line = line.strip()
    if line[0] != "\\":
        print(line)